# Call Center Analytics - Weekly Report

This notebook analyzes call center data to provide insights on:
- Average waiting and talking times by customer type
- Abandoned calls analysis
- Week-over-week comparisons

**Data Source:** CSV files in `recent_data/` directory
**Output:** Interactive HTML reports with visualizations

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import glob
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from thefuzz import fuzz

# Import custom utility functions from our module
from call_analytics_utils import (
    add_week_label,
    plot_avg_talking_grouped,
    plot_abandoned_by_day_of_week,
    plot_avg_waiting_time,
    get_week_date_label,
    hms_to_seconds,
    extract_phone_number
)

c:\ProgramData\Anaconda3\Lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Configuration

Set parameters for analysis here

In [2]:
# Configuration Parameters
# Customize these settings as needed

# Weeks to compare (1 = most recent, 2 = previous week)
WEEKS_TO_COMPARE = (2, 1)

# Minimum waiting time (seconds) to consider a call as abandoned
MIN_ABANDON_TIME_SECONDS = 20

# Customer types to analyze
CUSTOMER_TYPES = ("Retail", "Trade Customer")

# File patterns for data import
CALL_LOG_PATTERN = 'recent_data/CallLogthisYTD*.csv'
ABANDONED_CALLS_PATTERN = 'recent_data/AbandonedthisYTD*.csv'

# Output file paths
OUTPUT_HTML = "Southside_Call_Metrics.html"
OUTPUT_TALKING_HTML = "avg_talking.html"
OUTPUT_TALKING_PNG = "avg_talking.png"

print("Configuration loaded successfully")

Configuration loaded successfully


# Data Loading and Processing

## Step 1: Load Call Log Data

Import all call log CSV files and process them

In [3]:
# Find and load all call log CSV files
files = glob.glob(CALL_LOG_PATTERN)
print(f"Found {len(files)} call log file(s)")

# Read and concatenate all CSV files
call_reports = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

# Remove the totals row if present
if call_reports['Call Time'].iloc[-1] == 'Totals':
    call_reports = call_reports.iloc[:-1]

# Rename column for consistency
call_reports.rename(columns={"From": "Caller ID"}, inplace=True)
# Add week labels and process caller information
call_reports = add_week_label(call_reports)
call_reports.head()

Found 1 call log file(s)


,Call Time,Call ID,Caller ID,To,Direction,Status,Ringing,Talking,Cost,Call Activity Details,Type,week
0,2025-11-10 19:53:10,00000000-01dc-527b-9b99-ae85000012ee,0852255722,"Gorman, Joe (217)",Inbound,Answered,0,199,0.0,"Ended by Gorman, Joe (217)",Retail,3
1,2025-11-10 19:52:56,00000000-01dc-527b-9b99-ae85000012ee,0852255722,"Gerard, Tohill (204)",Inbound,Unanswered,14,0,0.0,"Gerard, Tohill (204) call was taken by Gorman,...",Retail,1
2,2025-11-10 19:52:56,00000000-01dc-527b-9b99-ae85000012ee,0852255722,Sales Queue (501),Inbound Queue,Waiting,0,14,0.0,"Inbound: WILSON, BEN (0852255722) → Via trunk:...",Retail,1
3,2025-11-10 19:32:13,00000000-01dc-5278-a2b0-e2ed000012ed,0894612821,"Gorman, Joe (217)",Inbound,Answered,0,88,0.0,"Ended by Gorman, Joe (217)",Retail,1
4,2025-11-10 19:32:10,00000000-01dc-5278-a2b0-e2ed000012ed,0894612821,"Leavy, Darragh (211)",Inbound,Unanswered,3,0,0.0,"Leavy, Darragh (211) call was taken by Gorman,...",Retail,1


In [4]:
call_reports.head(1).T

,0
Call Time,2025-11-10 19:53:10
Call ID,00000000-01dc-527b-9b99-ae85000012ee
Caller ID,0852255722
To,"Gorman, Joe (217)"
Direction,Inbound
Status,Answered
Ringing,0
Talking,199
Cost,0.0
Call Activity Details,"Ended by Gorman, Joe (217)"


In [5]:
call_reports["Type"].value_counts()

Type
Retail            19199
Trade Customer      801
Name: count, dtype: int64

In [6]:
trade_customer =call_reports[call_reports["Type"] == "Trade Customer"]
trade_customer['Direction'].value_counts()

Direction
Outbound         501
Internal         197
Inbound           53
Inbound Queue     50
Name: count, dtype: int64

In [12]:
call_reports[(call_reports["Type"] == "Trade Customer")&(call_reports['Direction']=='Inbound')]

,Call ID,Call Time,Caller ID,To,Direction,Status,Ringing,Talking,Cost,Call Activity Details,Type,week
43,00000000-01dc-3c2c-647a-5635000008f4,2025-10-13 11:30:41,anonymous,"Gerard, Tohill (204)",Inbound,Answered,14,125,0.0,"Ended by Gerard, Tohill (204)",Trade Customer,3
130,00000000-01dc-3c40-0314-ac2c0000094b,2025-10-13 13:53:05,anonymous,"David, Carey (209)",Inbound,Answered,5,155,0.0,Ended by Anonymous:Sales Main DID (anonymous),Trade Customer,3
447,00000000-01dc-3cfd-03c8-387200000a88,2025-10-14 12:23:50,anonymous,"Gerard, Tohill (204)",Inbound,Answered,4,89,0.0,Ended by Anonymous:Sales Main DID (anonymous),Trade Customer,3
624,00000000-01dc-3d27-6abc-b12d00000b39,2025-10-14 17:27:21,anonymous,"Leavy, Darragh (211)",Inbound,Answered,6,28,0.0,"Ended by Leavy, Darragh (211)",Trade Customer,3
627,00000000-01dc-3d28-9be4-856d00000b3c,2025-10-14 17:35:53,anonymous,"Leavy, Darragh (211)",Inbound,Answered,5,347,0.0,"Ended by Leavy, Darragh (211)",Trade Customer,3
1947,00000000-01dc-4033-5cd6-1ea300001064,2025-10-18 14:31:38,anonymous,"Mark, McEndoo (207)",Inbound,Answered,12,183,0.0,Ended by Anonymous:Sales Main DID (anonymous),Trade Customer,3
2103,00000000-01dc-40ea-8b4c-7d9800000033,2025-10-19 12:21:41,anonymous,"Leavy, Darragh (211)",Inbound,Answered,13,56,0.0,Ended by Anonymous:Sales Main DID (anonymous),Trade Customer,3
2112,00000000-01dc-40ed-306c-436c0000003c,2025-10-19 12:40:37,anonymous,"Leavy, Darragh (211)",Inbound,Answered,3,31,0.0,Ended by Anonymous:Sales Main DID (anonymous),Trade Customer,3
2734,00000000-01dc-4285-b008-25e1000002aa,2025-10-21 13:26:23,anonymous,"Pos 5, Sales (221)",Inbound,Answered,13,368,0.0,Ended by Anonymous:Sales Main DID (anonymous),Trade Customer,3
2740,00000000-01dc-4287-3787-1d47000002b0,2025-10-21 13:35:59,anonymous,"David, Carey (209)",Inbound,Answered,8,24,0.0,Ended by Anonymous:Sales Main DID (anonymous),Trade Customer,3


In [8]:
# Find and load all call log CSV files
files = glob.glob(CALL_LOG_PATTERN)
print(f"Found {len(files)} call log file(s)")

# Read and concatenate all CSV files
call_reports = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

# Remove the totals row if present
if call_reports['Call Time'].iloc[-1] == 'Totals':
    call_reports = call_reports.iloc[:-1]

# Rename column for consistency
call_reports.rename(columns={"From": "Caller ID"}, inplace=True)

# Add week labels and process caller information
call_reports = add_week_label(call_reports)

# Remove outbound and internal calls
#call_reports = call_reports[(call_reports['Direction'] != 'Outbound')&(call_reports['Direction'] != 'Internal')]

# Group by Call ID and aggregate relevant fields
call_reports = call_reports.groupby('Call ID').agg({'Call Time':'first',
                                                    'Caller ID':'first',
                                     'To':'first','Direction':'first',
                                     'Status':'first',
                                     'Ringing':'sum','Talking':'sum',
                                     'Cost':'sum', 'Call Activity Details':'first',
                                     'Type':'first','week':'first',
                              }).reset_index()

print(f"Loaded {len(call_reports)} call records")
print(f"Date range: {call_reports['Call Time'].min()} to {call_reports['Call Time'].max()}")

# Check the actual day of week for max date
max_date = pd.to_datetime(call_reports['Call Time']).max()
print(f"Max date day of week: {max_date.day_name()} (weekday={max_date.weekday()})")

# Show week boundaries
days_since_monday = max_date.weekday()
most_recent_monday = max_date - pd.Timedelta(days=days_since_monday)
print(f"\nWeek 1 Monday: {most_recent_monday.strftime('%Y-%m-%d %A')}")
print(f"Week 1 range: {most_recent_monday.strftime('%Y-%m-%d')} to {(most_recent_monday + pd.Timedelta(days=6)).strftime('%Y-%m-%d')}")
print(f"Week 2 range: {(most_recent_monday - pd.Timedelta(days=7)).strftime('%Y-%m-%d')} to {(most_recent_monday - pd.Timedelta(days=1)).strftime('%Y-%m-%d')}")

print(f"\nCustomer type distribution:")
print(call_reports['Type'].value_counts())

print(f"\nWeek distribution:")
print(call_reports['week'].value_counts().sort_index())

# Show some sample dates for each week
print(f"\n--- Sample dates by week ---")
for week_num in [1, 2, 3]:
    week_data = call_reports[call_reports['week'] == week_num]
    if len(week_data) > 0:
        print(f"Week {week_num}: {week_data['Call Time'].min()} to {week_data['Call Time'].max()} ({len(week_data)} records)")

# Check Status and Direction values
print(f"\n--- Status values ---")
print(call_reports['Status'].value_counts())

print(f"\n--- Direction values ---")
print(call_reports['Direction'].value_counts())

call_reports.head()

Found 1 call log file(s)
Loaded 9038 call records
Date range: 2025-10-13 10:10:49 to 2025-11-10 19:53:10
Max date day of week: Monday (weekday=0)

Week 1 Monday: 2025-11-10 Monday
Week 1 range: 2025-11-10 to 2025-11-16
Week 2 range: 2025-11-03 to 2025-11-09

Customer type distribution:
Type
Retail            8296
Trade Customer     742
Name: count, dtype: int64

Week distribution:
week
1    2227
2    2248
3    4563
Name: count, dtype: int64

--- Sample dates by week ---
Week 1: 2025-11-04 07:29:05 to 2025-11-10 19:32:13 (2227 records)
Week 2: 2025-10-28 07:49:30 to 2025-11-03 19:09:09 (2248 records)
Week 3: 2025-10-13 10:10:49 to 2025-11-10 19:53:10 (4563 records)

--- Status values ---
Status
Answered      7147
Unanswered    1891
Name: count, dtype: int64

--- Direction values ---
Direction
Inbound          6543
Inbound Queue    1800
Outbound          501
Internal          194
Name: count, dtype: int64


,Call ID,Call Time,Caller ID,To,Direction,Status,Ringing,Talking,Cost,Call Activity Details,Type,week
0,00000000-01dc-3c21-443c-404a000008c9,2025-10-13 10:10:49,014592069,"David, Carey (209)",Inbound,Answered,4,114,0.0,"Ended by TRUCK CENTRE,, MURPHY`S (014592069)",Retail,3
1,00000000-01dc-3c21-693b-e47d000008ca,2025-10-13 10:11:51,014593015,"Leavy, Darragh (211)",Inbound,Answered,4,75,0.0,"Ended by Leavy, Darragh (211)",Retail,3
2,00000000-01dc-3c21-6d74-a0cd000008cb,2025-10-13 10:12:03,016266152,"Gerard, Tohill (204)",Inbound,Answered,5,47,0.0,"Ended by Gerard, Tohill (204)",Retail,3
3,00000000-01dc-3c21-eef2-ff21000008cc,2025-10-13 10:15:35,014036700,"Leavy, Darragh (211)",Inbound,Answered,9,82,0.0,Ended by 014036700:Sales Main DID (014036700),Retail,3
4,00000000-01dc-3c22-3929-5730000008cd,2025-10-13 10:18:10,0877590979,"Leavy, Darragh (211)",Inbound,Answered,5,66,0.0,"Ended by Leavy, Darragh (211)",Retail,3


## Step 2: Analyze Talking Time

Generate visualization for average talking time by customer type

In [9]:
# Create talking time visualization comparing selected weeks
fig_talk = plot_avg_talking_grouped(
    call_reports,
    weeks=WEEKS_TO_COMPARE,
    save_html_path=OUTPUT_TALKING_HTML,
    save_png_path=OUTPUT_TALKING_PNG
)

fig_talk.show()

Retrying in 7.49 seconds due to HTTPConnectionPool(host='localhost', port=11317): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000015A1D371430>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))...


## Step 3: Process Abandoned Calls Data

Load and analyze abandoned calls

In [11]:
# Load trade customer data
trade_customers_list = pd.read_csv("recent_data/407N 13th November 2025.csv")

# Extract phone numbers from the three columns (excluding Customer column)
# Flatten all phone numbers into a single list and remove NaNs
trade_phone_numbers = []

for col in ['A/C Tel', 'Sls Tel', 'MobileNumber']:
    trade_customers_list[col] = trade_customers_list[col].str.replace(r"\D+", "", regex=True)
    # Get non-null values from each column
    numbers = trade_customers_list[col].dropna().astype(str).tolist()
    trade_phone_numbers.extend(numbers)

# Remove duplicates and convert to list
trade_phone_numbers = list(set(trade_phone_numbers))

print(f"Total trade phone numbers extracted: {len(trade_phone_numbers)}")
print(f"Sample numbers: {trade_phone_numbers[:5]}")

trade_customers_list

Total trade phone numbers extracted: 2719
Sample numbers: ['', '0863365918', '0868154750', '0862566880', '2966894']


,Customer,A/C Tel,Sls Tel,MobileNumber
0,123456789,NaN,NaN,NaN
1,555,NaN,NaN,0862553507
2,A001,016793722,0894241049,NaN
3,A002,016793722,0863589622,0852243519
4,AA000,014964366,NaN,0862633794
...,...,...,...,...
2539,XL001,6259500,6259500,NaN
2540,ZWAS001,NaN,014298539,NaN
2541,ZWP000,2950646,2950646,NaN
2542,ZWT000,2983153,2983153,0872560050


In [22]:
trade_phone_numbers

['',
 '0863365918',
 '0868154750',
 '0862566880',
 '2966894',
 '0873632524',
 '4517447',
 '0416852939',
 '0858369101',
 '0872074238',
 '0852885855',
 '0870930900',
 '0857020539',
 '0872609862',
 '405135155',
 '0879033425',
 '0868282589',
 '061379903',
 '014123993',
 '0879290634',
 '0877630653',
 '0876607766',
 '012545245',
 '0851686333',
 '4502800',
 '0876944531',
 '012959907',
 '018829300',
 '0879049225',
 '018321045',
 '0861208719',
 '0868674085',
 '0526187414',
 '0876870022',
 '4909550',
 '0876903593',
 '0857465281',
 '4973536',
 '60085144569',
 '0857806635',
 '6613344085',
 '0879946981',
 '0899486780',
 '0863836199',
 '018855554',
 '0872713189',
 '0876679518',
 '087929660',
 '0851445692',
 '0872577907',
 '0872332323',
 '0863281956',
 '0894712865',
 '0863124474',
 '014973303',
 '0830022210',
 '0872558536',
 '0863270528',
 '0862285214',
 '85559918554',
 '012176777',
 '4194500',
 '018441996',
 '018242626',
 '0146401606',
 '014698888',
 '0863135748',
 '0879198270',
 '4598444',
 '087284

In [ ]:
# Optional: View trade customers data for verification
trade_customers_check = call_reports[call_reports['Type'] == 'Trade Customer'].reset_index(drop=True)
print(f"Total trade customer calls: {len(trade_customers_check)}")
print(f"Week distribution:")
print(trade_customers_check['week'].value_counts().sort_index())
trade_customers_check.head()

Total trade customer calls: 103
Week distribution:
week
1    36
2    33
3    34
Name: count, dtype: int64


,Call Time,Call ID,Caller ID,To,Direction,Status,Ringing,Talking,Cost,Call Activity Details,Type,week
0,2025-11-10 14:55:39,00000000-01dc-5251-8904-87bc00001288,anonymous,"David, Carey (209)",Inbound,Answered,9,147,0.0,"Ended by David, Carey (209)",Trade Customer,1
1,2025-11-10 14:51:45,00000000-01dc-5251-8904-87bc00001288,anonymous,Sales Queue (501),Inbound Queue,Waiting,0,242,0.0,Inbound: Anonymous:Sales Main DID (anonymous) ...,Trade Customer,1
2,2025-11-09 17:24:51,00000000-01dc-519d-c14b-81590000118b,anonymous,Voice Agent,Inbound,Answered,0,4,0.0,Ended by Anonymous:Sales Main DID (anonymous),Trade Customer,1
3,2025-11-09 17:24:51,00000000-01dc-519d-c14b-81590000118b,anonymous,Sales Out of Office IVR (801),Inbound,Unanswered,0,0,0.0,"Sales Out of Office IVR (801) → Out of office,...",Trade Customer,1
4,2025-11-09 17:24:50,00000000-01dc-519d-c14b-81590000118b,anonymous,Sales Queue (501),Inbound Queue,Unanswered,0,0,0.0,Inbound: Anonymous:Sales Main DID (anonymous) ...,Trade Customer,1


### Map Customer Types and Talk Times

Match abandoned calls with customer information from call reports

In [18]:
# Load abandoned calls data
files = glob.glob(ABANDONED_CALLS_PATTERN)
print(f"Found {len(files)} abandoned calls file(s)")

hold_calls = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

# Group by call time and caller ID to get unique abandoned calls
abandoned_calls = hold_calls.groupby(['Call Time', 'Caller ID']).agg({
    'Waiting Time': 'first',
    'Queue': 'first'
}).reset_index()

# Process the data
abandoned_calls['Call Time'] = pd.to_datetime(abandoned_calls['Call Time'])
abandoned_calls["Waiting Time"] = pd.to_numeric(
    abandoned_calls["Waiting Time"].apply(hms_to_seconds),
    errors="coerce"
)

# Filter for calls with significant wait time
abandoned_calls = abandoned_calls[
    abandoned_calls['Waiting Time'] > MIN_ABANDON_TIME_SECONDS
].reset_index(drop=True)

# Add placeholder columns for processing
abandoned_calls['Sentiment'] = 0
abandoned_calls['Summary'] = 0
abandoned_calls['Transcription'] = 0
abandoned_calls['Talking'] = 0
abandoned_calls['Ringing'] = 0

# Add week labels and process
abandoned_calls = add_week_label(abandoned_calls)
abandoned_calls.drop(
    columns=['Talking', 'Ringing', 'Queue'],
    inplace=True
)

print(f"Total abandoned calls (>{MIN_ABANDON_TIME_SECONDS}s wait): {len(abandoned_calls)}")
print(f"Week distribution:")
print(abandoned_calls['week'].value_counts().sort_index())

Found 1 abandoned calls file(s)
Total abandoned calls (>20s wait): 1222
Week distribution:
week
1    286
2    432
3    504
Name: count, dtype: int64


In [20]:
# Map Customer Type using client list 
def is_fuzzy_match(x, items, threshold=80):
    x = str(x)
    return any(fuzz.ratio(x, str(item)) >= threshold for item in items)

# Map the customer type to abandoned calls using Call Time first
abandoned_calls['Type'] = abandoned_calls['Call Time'].apply(lambda x: "Trade Customer" if is_fuzzy_match(x, trade_phone_numbers) else "Retail")

print(f"\nAbandoned calls by type (after mapping with trade phone numbers):")
print(abandoned_calls['Type'].value_counts())

print(f"\nTrade customer abandoned calls: {(abandoned_calls['Type'] == 'Trade Customer').sum()}")

# Show a sample of the mapped data
print(f"\nSample of mapped abandoned calls:")
abandoned_calls.head(10)


Abandoned calls by type (after mapping with trade phone numbers):
Type
Retail    1222
Name: count, dtype: int64

Trade customer abandoned calls: 0

Sample of mapped abandoned calls:


,Call Time,Caller ID,Waiting Time,Type,week
0,2025-10-17 18:29:57,0874406912,56,Retail,3
1,2025-10-17 18:30:11,0838473134,63,Retail,3
2,2025-10-17 18:31:29,0851862266,38,Retail,3
3,2025-10-17 18:32:03,0857628372,454,Retail,3
4,2025-10-17 18:37:41,0834630513,96,Retail,3
5,2025-10-17 18:38:47,0876136300,22,Retail,3
6,2025-10-17 18:39:15,0876136300,24,Retail,3
7,2025-10-17 18:44:46,0876136300,190,Retail,3
8,2025-10-17 18:44:47,0879346997,390,Retail,3
9,2025-10-17 18:52:18,0858221459,185,Retail,3


In [26]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, str(a), str(b)).ratio()

abandoned_calls['Type']  = abandoned_calls['Type'].apply(
    lambda x: "Trade Customer"
    if any(similar(x, item) >= 0.8 for item in trade_phone_numbers)
    else "Retail"
)

In [27]:
abandoned_calls.Type.value_counts()

Type
Retail    1222
Name: count, dtype: int64

In [29]:
call_reports['Type'] = call_reports['Type'].apply(
    lambda x: "Trade Customer"
    if any(similar(x, item) >= 0.8 for item in trade_phone_numbers)
    else "Retail"
)

In [30]:
call_reports['Type'].value_counts()

Type
Retail    19294
Name: count, dtype: int64

In [9]:
# Diagnostic: Understand the abandoned calls data
print("=" * 70)
print("ABANDONED CALLS DIAGNOSTIC")
print("=" * 70)

print(f"\nTotal abandoned calls (>20s wait): {len(abandoned_calls)}")
print(f"\nBreakdown by week:")
print(abandoned_calls['week'].value_counts().sort_index())


print(f"\n--- Week 1 Breakdown ---")
week1_abandoned = abandoned_calls[abandoned_calls['week'] == 1]
print(f"Total Week 1 records: {len(week1_abandoned)}")

print(f"\n--- Week 2 Breakdown ---")
week2_abandoned = abandoned_calls[abandoned_calls['week'] == 2]
print(f"Total Week 2 records: {len(week2_abandoned)}")

print(f"\n--- Customer Type Distribution (ALL abandoned records) ---")
print(abandoned_calls.groupby(['week', 'Type']).size().unstack(fill_value=0))

print("=" * 70)

ABANDONED CALLS DIAGNOSTIC

Total abandoned calls (>20s wait): 1222

Breakdown by week:
week
1    286
2    432
3    504
Name: count, dtype: int64

--- Week 1 Breakdown ---
Total Week 1 records: 286

--- Week 2 Breakdown ---
Total Week 2 records: 432

--- Customer Type Distribution (ALL abandoned records) ---
Type  Retail  Trade Customer
week                        
1        260              26
2        394              38
3        456              48


In [10]:
# Verify trade customer mapping worked
trade_abandoned = abandoned_calls[abandoned_calls['Type'] == 'Trade Customer']

if len(trade_abandoned) > 0:
    print(f"\n✓ Successfully mapped {len(trade_abandoned)} trade customer abandoned calls")
    print(f"\nWeek distribution for trade customer abandoned calls:")
    print(trade_abandoned['week'].value_counts().sort_index())
    print(f"\nSample of trade customer abandoned calls:")
    trade_abandoned.head()
else:
    print("\n⚠ Warning: No trade customer abandoned calls found after mapping")


✓ Successfully mapped 112 trade customer abandoned calls

Week distribution for trade customer abandoned calls:
week
1    26
2    38
3    48
Name: count, dtype: int64

Sample of trade customer abandoned calls:


## Step 4: Average Waiting Time Analysis

Visualize average waiting time for abandoned calls

In [11]:
# Generate average waiting time plot for abandoned calls
fig_waiting = plot_avg_waiting_time(
    df=abandoned_calls,
    waiting_col="Waiting Time",
    type_col="Type",
    week_col="week",
    types=CUSTOMER_TYPES,
    weeks=WEEKS_TO_COMPARE,
    title="Average Waiting Time (minutes) - Abandoned Calls"
)

fig_waiting.show()

## Step 5: Generate Combined Report

Create a comprehensive HTML report combining all visualizations

In [12]:
# Generate abandoned calls by day of week plot
fig_abandoned = plot_abandoned_by_day_of_week(
    abandoned_calls,
    weeks=WEEKS_TO_COMPARE,
    all_calls_df=call_reports  # Pass all calls data for total/answered statistics
)

fig_abandoned.show()

In [13]:
# Create combined figure with all three visualizations
combined_fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=(
        "Abandoned Calls - Average Waiting Time",
        "Average Talking Time",
        "Abandoned Calls by Day of Week"
    ),
    specs=[[{}], [{}], [{}]],  # One plot per row
    vertical_spacing=0.08,
    row_heights=[0.33, 0.33, 0.33]
)

# Add waiting time plot (row 1) - hide legend for this plot
for i, trace in enumerate(fig_waiting.data):
    trace.showlegend = False  # Hide legend from plot 1
    combined_fig.add_trace(trace, row=1, col=1)

# Add talking time plot (row 2) - show legend for this plot only
for trace in fig_talk.data:
    trace.showlegend = False  # Hide legend from plot 1
    combined_fig.add_trace(trace, row=2, col=1)

# Add abandoned by day plot (row 3) - hide legend
for trace in fig_abandoned.data:
    trace.showlegend = False  # Hide duplicate legend
    combined_fig.add_trace(trace, row=3, col=1)

# Calculate dynamic y-position for annotation in the third plot
abandoned_filtered = abandoned_calls[abandoned_calls['week'].isin(WEEKS_TO_COMPARE)].copy()
abandoned_filtered['day_of_week'] = pd.to_datetime(
    abandoned_filtered['Call Time']
).dt.day_name()
day_counts = abandoned_filtered.groupby(['week', 'day_of_week']).size().reset_index(name='count')

if len(day_counts) > 0:
    max_count = day_counts['count'].max()
    annotation_y = max_count * 0.8  # Position at 80% of max height
else:
    annotation_y = 10  # Default fallback


# Add custom text labels to bars in plots 1 and 2
# Loop through traces and set text based on trace name
for trace in combined_fig.data:
    if trace.type == 'bar' and hasattr(trace, 'name'):
        # Extract week number from trace name (e.g., "Retail - Week 1" -> "Week 1")
        if 'Week' in str(trace.name):
            week_label = str(trace.name).split(' - ')[-1] if ' - ' in str(trace.name) else str(trace.name)
            # Set text to appear on each bar
            trace.text = [week_label] * len(trace.x)
            trace.textposition = 'inside'
            trace.textfont = dict(color='white', size=12)

# Get max date from abandoned calls for week label calculation
max_date = pd.to_datetime(abandoned_calls['Call Time']).max()

# Calculate statistics for annotation
week1_data = abandoned_filtered[abandoned_filtered['week'] == 1]
week2_data = abandoned_filtered[abandoned_filtered['week'] == 2]

avg_time_week1 = week1_data['Waiting Time'].mean() / 60 if len(week1_data) > 0 else 0
avg_time_week2 = week2_data['Waiting Time'].mean() / 60 if len(week2_data) > 0 else 0
count_week1 = len(week1_data)
count_week2 = len(week2_data)

# Add summary annotation with max_date passed to get_week_date_label
annotation_text = (
    f"<b>Avg Time to Abandon:</b><br>"
    f"Week {get_week_date_label(1, max_date)}: {avg_time_week1:.1f}m ({count_week1} calls)<br>"
    f"Week {get_week_date_label(2, max_date)}: {avg_time_week2:.1f}m ({count_week2} calls)"
)

combined_fig.add_annotation(
    text=annotation_text,
    xref="x3", yref="y3",  # Reference third subplot
    x=-0.4, y=annotation_y,
    showarrow=False,
    align="left",
    xanchor="left",
    yanchor="middle",
    bgcolor="rgba(255, 255, 255, 0.8)",
    bordercolor="black",
    borderwidth=1,
    font=dict(size=10)
)


# Update axis labels
combined_fig.update_yaxes(title_text="Minutes", row=1, col=1)
combined_fig.update_yaxes(title_text="Minutes", row=2, col=1)

# Update layout
combined_fig.update_layout(
    title_text="Southside Call Center Metrics",
    title_x=0.5,
    showlegend=True,
    barmode="group",
    height=1200
)

# Save to HTML
combined_fig.write_html(OUTPUT_HTML)
print(f"\n✓ Report saved to: {OUTPUT_HTML}")
print(f"✓ Talking time chart saved to: {OUTPUT_TALKING_HTML}")

combined_fig.show()


✓ Report saved to: Southside_Call_Metrics.html
✓ Talking time chart saved to: avg_talking.html


In [16]:
call_reports[call_reports['week'] == 1]['Status'].value_counts()

Status
Answered      2287
Waiting       1652
Unanswered    1212
Name: count, dtype: int64

In [20]:
abandoned_calls[abandoned_calls['week'] == 1]

,Call Time,Caller ID,Waiting Time,Type,week
935,2025-11-04 08:47:27,0872458861,91,Retail,1
936,2025-11-04 08:50:10,0868767882,42,Retail,1
937,2025-11-04 08:50:24,0861922597,23,Retail,1
938,2025-11-04 08:54:48,0894201591,26,Retail,1
939,2025-11-04 08:55:24,0894201591,277,Retail,1
...,...,...,...,...,...
1216,2025-11-10 15:46:31,014633500,129,Trade Customer,1
1217,2025-11-10 17:58:36,0894466314,468,Retail,1
1218,2025-11-10 18:21:52,0852810616,92,Retail,1
1219,2025-11-10 18:39:17,0874917109,29,Retail,1


In [19]:
abandoned_calls[abandoned_calls['Caller ID'] == '0894612821']

,Call Time,Caller ID,Waiting Time,Type,week


In [21]:
abandoned_calls

,Call Time,Caller ID,Waiting Time,Type,week
0,2025-10-17 18:29:57,0874406912,56,Retail,3
1,2025-10-17 18:30:11,0838473134,63,Retail,3
2,2025-10-17 18:31:29,0851862266,38,Retail,3
3,2025-10-17 18:32:03,0857628372,454,Retail,3
4,2025-10-17 18:37:41,0834630513,96,Retail,3
...,...,...,...,...,...
1217,2025-11-10 17:58:36,0894466314,468,Retail,1
1218,2025-11-10 18:21:52,0852810616,92,Retail,1
1219,2025-11-10 18:39:17,0874917109,29,Retail,1
1220,2025-11-10 18:47:35,0892547705,27,Retail,1




Add legend to both plots (see if they can be level with plots) or add red “this week” and blue ‘Last week”

Abandoned calls
Add to hover (minimum, average (MODE), maximum)
Total number calls, Number of calls answered, number of calls abandoned 
